## Create Mike's dictionary of Morton codes with start and end points for the 512^3

In [15]:
desired_cube_side = 512
chunk_size = 64
array_cube_side = 2048

raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt'
use_dask = True # Has issues on SciServer Compute
dest_folder_name = "sabl2048b" # B is the high-rate data
write_type = "prod" # or "back"
timestep_nr = 1
# timestep_range = range(1) # Ned's new High-rate fixed-dt has only 5 timesteps

In [2]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing

/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing


In [3]:
import xarray as xr
from utils import write_tools
import os

### Get target Folder list

<font color="orange">Don't delete the CD cell!</font>

In [5]:
%cd /home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt

/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt


In [6]:
merged_velocity = xr.open_dataset(raw_ncar_folder_path + "/jhd." + str(timestep_nr).zfill(3) + ".nc")

In [8]:
# Split 2048^3 into smaller 512^3 arrays
dims = [dim for dim in merged_velocity.dims]
dims.reverse() # use (nnz, nny, nnx) instead of (nnx, nny, nnz)

smaller_groups, range_list = write_tools.split_zarr_group(merged_velocity, desired_cube_side, dims)

In [39]:
range_list[:4]

[[[0, 512], [0, 512], [0, 512]],
 [[0, 512], [0, 512], [512, 1024]],
 [[0, 512], [0, 512], [1024, 1536]],
 [[0, 512], [0, 512], [1536, 2048]]]

In [10]:
len(range_list)

64

In [11]:
# Given up in favor of Ryan's node coloring technique
#     z_order = write_tools.morton_order_cube(cube_side=4)

node_assignments = write_tools.node_assignment(cube_side=4)

In [40]:
mike_dict = {}
for i in range(len(range_list)):            
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]
    
#     print(max_coord)
            
    mike_dict[dest_folder_name + str(i + 1).zfill(2)] = \
        (write_tools.morton_pack(array_cube_side, min_coord[2], min_coord[1], min_coord[0]), write_tools.morton_pack(array_cube_side, max_coord[2], max_coord[1], max_coord[0]))
    # Ordered 2-1-0 bcs. dims are nnz-nny-nnx
    
#     print(mike_dict[dest_folder_name + str(i + 1).zfill(2)])

In [41]:
mike_dict

{'sabl2048b01': (0, 134217727),
 'sabl2048b02': (134217728, 268435455),
 'sabl2048b03': (1073741824, 1207959551),
 'sabl2048b04': (1207959552, 1342177279),
 'sabl2048b05': (268435456, 402653183),
 'sabl2048b06': (402653184, 536870911),
 'sabl2048b07': (1342177280, 1476395007),
 'sabl2048b08': (1476395008, 1610612735),
 'sabl2048b09': (2147483648, 2281701375),
 'sabl2048b10': (2281701376, 2415919103),
 'sabl2048b11': (3221225472, 3355443199),
 'sabl2048b12': (3355443200, 3489660927),
 'sabl2048b13': (2415919104, 2550136831),
 'sabl2048b14': (2550136832, 2684354559),
 'sabl2048b15': (3489660928, 3623878655),
 'sabl2048b16': (3623878656, 3758096383),
 'sabl2048b17': (536870912, 671088639),
 'sabl2048b18': (671088640, 805306367),
 'sabl2048b19': (1610612736, 1744830463),
 'sabl2048b20': (1744830464, 1879048191),
 'sabl2048b21': (805306368, 939524095),
 'sabl2048b22': (939524096, 1073741823),
 'sabl2048b23': (1879048192, 2013265919),
 'sabl2048b24': (2013265920, 2147483647),
 'sabl2048b25':

In [44]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing

/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing


In [45]:
import dill

pickle_file = './filename_morton_ranges.pickle'

# the dictionary that will be written to the pickle file. empty dictionary in this example, but you would fill it with the data.
filename_morton_map = mike_dict

with open(pickle_file, 'wb') as pickled_filepath:
    dill.dump(filename_morton_map, pickled_filepath)